# Grade: /100 points

# Assignment 01: Supervised learning, Linear models, and Loss functions

In this assignment, you're going to write your own methods to fit a linear model using either an OLS or LAD cost function.  

## Data set 

For this assignment, we will examine some data representing possums in Australia and New Guinea. The data frame contains 46 observations on the following 6 variables:

* sex: Sex, either m (male) or f (female).
* age: Age in years.
* headL: Head length, in mm.
* skullW: Skull width, in mm.
* totalL: Total length, in cm.
* tailL: Tail length, in cm.

## Follow These Steps Before Submitting
Once you are finished, ensure to complete the following steps.

1.  Restart your kernel by clicking 'Kernel' > 'Restart & Run All'.

2.  Fix any errors which result from this.

3.  Repeat steps 1. and 2. until your notebook runs without errors.

4.  Submit your completed notebook to OWL by the deadline.




#### Before you start...

Recall that L1 loss function (sum of magnitudes, used for LAD model):

$$L_1(\theta) = \sum_{i=1}^{n} \lvert {y_i-\hat{y_i}} \rvert$$

L2 loss function (RSS, residual sum of squares, used for OLS model):

$$L_2(\theta) = \sum_{i=1}^{n} ({y_i-\hat{y_i}})^2$$

## Preliminaries

In [70]:
# Import all the necessary packages: 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy.stats as ss 
import scipy.optimize as so
from sklearn import linear_model
from sklearn.metrics import r2_score
%matplotlib inline


## Part 1
### Question 1.1:  /10 points


Read in the `possum.csv` file as a `pandas.DataFrame`.  Investigate the relationship between the possum's age and its tail length by plotting a scatter plot of the `age` and `tailL` columns. Add an `alpha`(transparency of the plotted dots) in case some data are overlapping. 

In [71]:
# Read in the data with pandas
possum_data = pd.read_csv("./possum.csv")

# Make the scatter plot (don't forget the axis labels)
possum_data_age = possum_data['age']
possum_data_tailL = possum_data['tailL']
plt.scatter(possum_data_age, possum_data_tailL, alpha= 0.8)
plt.xlabel("Age")
plt.ylabel("Tail Length")
plt.title("Relationship between age and Tail Length")
plt.show()

### Question 1.2: /5 point

Recall that the linear model, we obtain predictions by computing 

$$ \hat{\mathbf{y}} = \mathbf{X} \hat{\beta} $$

Here, $\mathbf{X}$ is a design matrix which includes a column of ones, $\hat{\beta}$ are coefficients, and $\hat{\mathbf{y}}$ are outcomes.  Write a function `linearModelPredict` to compute linear model predictions given data and a coefficient vector.  The function should take as it's arguments a 1d-array of coefficients `b` and the design matrix `X` as a 2d-array and return linear model predictions `yp`.

Test the function by setting 

```
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
```
and call your function with these values! 

Report $\hat{\mathbf{y}}$. 
What is the dimensionality of the numpy-array that you get back? 

Hint:  Read the documentation for `np.dot` or the `@` operator in `numpy`.

In [72]:
def linearModelPredict(b,X):
    yp = X @ b
    return yp

# Always important: Test the new function you have written! 
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
test_value = linearModelPredict(b, X)
print("Test the new function:")
print(test_value)
# By the way: What happens when b is a 2d-array? 
b_2D = np.array([[0.1,0.3], [0.1,0.3]])
test_2dArray = linearModelPredict(b_2D, X)
print("Test when b is a 2d-array:")
print(test_2dArray)

Test the new function:
[ 0.1 -0.2  0.7]
Test when b is a 2d-array:
[[0.1 0.3]
 [0.  0. ]
 [0.3 0.9]]


### Question 1.3: /15 points

Write a function `linearModelLossRSS` which computes and returns the loss function for an OLS model parameterized by $\beta$, as well as the gradient of the loss.  The function should take as its first argument a 1d-array `beta` of coefficients for the linear model, as its second argument the design matrix `X` as a 2d-array, and as its third argument a 1d-array `y` of observed outcomes.

Test the function with the values 

```
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
y = np.array([0,0.4,2]) 
```

Report the loss and the gradient. 

**Written answer**: To minimize the cost do you need increase or decrease the value of the parameters? 

In [73]:
def linearModelLossRSS(b,X,y):
    y_pred = linearModelPredict(b,X)
    residual = y - y_pred
    residual_sum_of_squares = sum(residual**2)
    #compute the gradient
    gradient = -2*np.dot(np.transpose(X),residual)
    return (residual_sum_of_squares, gradient)

# Testing
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
y = np.array([0,0.4,2])
loss, gradient = linearModelLossRSS(b,X,y)

print("Loss is : %.2f" % loss)
print("Gradient is :", gradient)

Loss is : 2.06
Gradient is : [-3.6 -4. ]


**Written answer**: To minimize the cost do you need increase or decrease the value of the parameters? 

I need to decrease the value of the parameters.

### Question 1.4:  /15 points. 

Now that you've implemented a loss function in question 1.3, it is now time to minimize it!

Write a function `linearModelFit` to fit a linear model.  The function should take as its first argument the design matrix `X` as a 2d-array, as its second argument a 1d-array `y` of outcomes, and as its third argument a function  `lossfcn` which returns as a tuple the value of the loss, as well as the gradient of the loss. As a result, it should return the estimated betas and the R2. 

**Hint**: Using `scipy.optimize.minimize` to minimize the customized loss function


Test the function with the values: 
```
X = np.array([[1,0],[1,-1],[1,2]])
y = np.array([0,0.4,2]) 
```

Report best parameters and the fitted R2 


In [74]:
def linearModelFit(X,y,lossfcn = linearModelLossRSS):
    nrows, ncols = X.shape
    b = np.zeros((ncols, 1))
    
    RESULT = so.minimize(lossfcn, b, args=(X, y),jac=True)
    estimated_betas = RESULT.x
    
    TSS = sum((y - np.mean(y))**2)
    RSS, deriv = linearModelLossRSS(estimated_betas, X, y)
    R2 = 1-(RSS / TSS)
    return (estimated_betas, R2)

# Testing:
X = np.array([[1,0],[1,-1],[1,2]])
y = np.array([0,0.4,2]) 
estimated_betas, R2 = linearModelFit(X,y)
print("Estimate beta is ", estimated_betas)
print("R2 is ", R2)

Estimate beta is  [0.6 0.6]
R2 is  0.75


/var/folders/fy/p355vwwd683668y784t68b2c0000gn/T/ipykernel_32562/3706014206.py:5: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  RESULT = so.minimize(lossfcn, b, args=(X, y),jac=True)


### Question 1.5: /15 points

Use the above functions to fit your model to the possum data. Then use your model and the fitted parameters to make predictions along a grid of equally spaced possum ages.  

**Hint** : Don't forget to include a column of ones in your design matrix to allow bias

Plot the data and add a line for the predicted values. You can get these by generating a new X-matrix with equally space ages (using for example np.linspace). Also report the R2 value for the fit. You can do this by either printing out the R2 of the fit or putting it on your plot via the `annotate` function in matplotlib.


R2 is  0.22286587947604142


/var/folders/fy/p355vwwd683668y784t68b2c0000gn/T/ipykernel_32562/3706014206.py:5: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  RESULT = so.minimize(lossfcn, b, args=(X, y),jac=True)


In [75]:
# Make the design matrix using np.c_ 
X = np.c_[np.ones(len(possum_data)), possum_data["age"].to_numpy()]
y = possum_data["tailL"].to_numpy()

# Call your fitting function 
betas, R2 = linearModelFit(X,y)

# Create the scatter plot (see question 1.1)
possum_data_age = possum_data['age']
possum_data_tailL = possum_data['tailL']
plt.scatter(possum_data_age, possum_data_tailL, alpha= 0.8)

# Create a new X matrix with equally space data 
y_pred = linearModelPredict(betas, X)
# Add the line to the graph 
plt.plot(X[:,1],y_pred, color='red', linestyle='--')
plt.title("Relationship between age and Tail Length (RSS)")
plt.xlabel("Age [years]")
plt.ylabel("Tail Length [cm]")
# Report R2 
print("R2 is " , R2)

## Part 2: LAD Regression

### Question 2.1:  /15 points

In the previous section, we worked with the squared loss.  Now, we'll implement a linear model with least absolute deviation loss.

Write a function `linearModelLossLAD` which computes the least absolute deviation loss function for a linear model  parameterized by $\beta$, as well as the gradient of the loss.  The function should take as its first argument a 1d-array `beta` of coefficients for the linear model, as its second argument the design matrix `X` as a 2d-array, and as its third argument a 1d-array `y` of observed outcomes.

Test the function with the values 

```
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
y = np.array([0,0.4,2]) 
```

Report the loss and the gradient. 

In [76]:
def linearModelLossLAD(b,X,y):
    y_pred = linearModelPredict(b, X)
    residual = y - y_pred
    sum_abs_dev = sum(abs(residual))
    grad = -np.dot(np.sign(residual), X)
    return (sum_abs_dev, grad)

## Testing
X = np.array([[1,0],[1,-1],[1,2]])
b = np.array([0.1,0.3])
y = np.array([0,0.4,2]) 

sum_abs_dev, grad = linearModelLossLAD(b,X,y)
print("Loss is : %.2f" % sum_abs_dev)
print("Gradient is :", grad)


Loss is : 2.00
Gradient is : [-1. -1.]


### Question 2.2: /10 points


Use the above functions to fit your LAD model. Use your model to make predictions along a grid of equally spaced possum ages.  Once fit, add the fitted line to the scatter plot as in question 1.5.  Also report the R2-value. 

**Written answer**: What is the difference in the fit obtained with an L1 as compared to the L2 cost function? Which one has a higher R2 value? Why?  

Note: If you recieve an error from the optimizer, it may be because the loss function for the LAD model is not differentiable at its minimum.  This will lead to some gradient based optimizers to fail to converge.  If this happens to you then pass `method="Powell"` to `scipy.optimize.minimize`.



/var/folders/fy/p355vwwd683668y784t68b2c0000gn/T/ipykernel_32562/3706014206.py:5: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  RESULT = so.minimize(lossfcn, b, args=(X, y),jac=True)


R2 is 0.1292302443520853


In [77]:
# Make the design matrix using np.c_ 
X = np.c_[np.ones(len(possum_data)), possum_data["age"].to_numpy()]
y = possum_data["tailL"].to_numpy()

# Call your fitting function 
betas, R2 = linearModelFit(X,y, linearModelLossLAD)

# Create the scatter plot (see question 1.1)
possum_data_age = possum_data['age']
possum_data_tailL = possum_data['tailL']
plt.scatter(possum_data_age, possum_data_tailL, alpha= 0.8)

# Create a new X matrix with equally space data 
y_pred = linearModelPredict(betas, X)
# Add the line to the graph 
plt.plot(X[:,1],y_pred, color='red', linestyle='--')
plt.title("Relationship between age and Tail Length (LAD)")
plt.xlabel("Age [years]")
plt.ylabel("Tail Length [cm]")
# Report R2 
print("R2 is" , R2)

**Written answer**: 
The OLS fit care more about the large values and hence it give more weight to the  outlier (9,55). 
The LAD's R2 value is lower
This is because OLS minimized the RSS, and therefore maximizes R2.  

### Question 2.3: /15 points

Fit an OLS model to the possum data with the `linear_model` module from the `sklearn` package by using the `LinearRegression` class.  In no more than two sentences, comment on the rsquared values from `sklearn` and the rsquared values from your models. Are they similar?

In [78]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [79]:
ypred = regr.predict(X)
print("R2: ", r2_score(y, ypred)) # this is the same value compared to my OLS model

R2:  0.22286587947604108
